In [1]:
#All Packages Used
import pandas as pd
import h2o
import numpy as np
from h2o.estimators.random_forest import H2ORandomForestEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
#Import Data
data = pd.read_csv('train.csv',parse_dates=True,index_col='Date')
store = pd.read_csv('store.csv')
states = pd.read_csv('store_states.csv')

/Users/LiamRoberts/anaconda3/envs/DS1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Drop NaN Columns
store.drop(columns = ['CompetitionOpenSinceMonth',
                      'CompetitionOpenSinceYear',
                      'Promo2SinceWeek',
                      'Promo2SinceYear'],inplace = True)

#Convert Date Columns
def add_dates(data):
    data['Year'] = data.index.year
    data['Month'] = data.index.month
    data['Day'] = data.index.day
    data['WeekOfYear'] = data.index.weekofyear
    data['MonthIndex'] = data.index.month + 12*(data.index.year-2013)
    data['WeekIndex'] = data.index.weekofyear + 52*(data.index.year-2013)
    return data

data = add_dates(data)

In [4]:
#Merge training dataframe
pd_df = data.merge(store,on='Store')
pd_df = pd_df.merge(states,on='Store')

#Drop times when store didnt make enough sales (these will also cause NaN values for log transforms)
pd_df = pd_df[(pd_df.Sales>0)]

In [5]:
pd_df.head(1)

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,MonthIndex,WeekIndex,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,State
0,1,5,5263,555,1,1,0,1,2015,7,31,31,31,135,c,a,1270.0,0,NaN,HE


In [6]:
#Adding store aggregate info
agg = pd_df[['Store','Customers','Sales']].groupby(by = 'Store').sum()
means = pd_df[['Store','Customers','Sales']].groupby(by = 'Store').mean()

promo = pd_df[(pd_df.Promo == 1)].groupby(by = 'Store').sum()
nopromo = pd_df[pd_df.Promo ==0].groupby(by = 'Store').sum()
holiday = pd_df[(pd_df.SchoolHoliday == 1) | (pd_df.StateHoliday == 1)].groupby(by = 'Store').sum()
noholiday = pd_df[(pd_df.SchoolHoliday == 0) & (pd_df.StateHoliday == 0)].groupby(by = 'Store').sum()
agg['salespercust'] = agg.Sales/agg.Customers
agg['promoratio'] = promo.Sales/nopromo.Sales
agg['holidayratio'] = holiday.Sales/noholiday.Sales
agg[['avgcustomers','avgsales']] = means[['Customers','Sales']]
agg.drop(columns = ['Sales','Customers'],inplace=True)

In [8]:
agg.head(1)

,salespercust,promoratio,holidayratio,avgcustomers,avgsales
Store,,,,,
1,8.437366,0.996361,2.289436,564.049936,4759.096031


In [9]:
pd_df = pd_df.merge(agg,on = 'Store')
pd_df.head(1)

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,...,Assortment,CompetitionDistance,Promo2,PromoInterval,State,salespercust,promoratio,holidayratio,avgcustomers,avgsales
0,1,5,5263,555,1,1,0,1,2015,7,...,a,1270.0,0,NaN,HE,8.437366,0.996361,2.289436,564.049936,4759.096031


In [196]:
#Adding Time Series Sales Info
def month_info(df,monthsback,joiner,how = 'inner'):
    TS_M = df[['MonthIndex','Sales','Store','Customers']].copy()
    TS_M['MonthIndex'] = TS_M['MonthIndex']+monthsback
    TS_M = TS_M.groupby(by = ['Store','MonthIndex']).median()
    TS_M.rename(columns = {'Sales':'Sales{}MonthsBack'.format(month),'Customers':'Customers{}MonthsBack'.format(month)},inplace=True)
    df = joiner.merge(TS_M,how = how,on=['Store','MonthIndex'])
    return df

def year_info(df,yearsback,joiner,how = 'inner'):
    TS_y = df[['Year','Sales','Store','Customers']].copy()
    TS_y['Year'] = TS_y['Year']+yearsback
    TS_y = TS_y.groupby(by = ['Store','Year']).median()
    TS_y.rename(columns = {'Sales':'Sales{}YearsBack'.format(month),'Customers':'Customers{}YearsBack'.format(month)},inplace=True)
    df = joiner.merge(TS_y,how = how,on=['Store','Year'])
    
months = [2,3,4]
for month in months:
    pd_df = month_info(pd_df,month,pd_df)

In [197]:
valid = pd_df[(pd_df.MonthIndex > 29)]
data = pd_df[(pd_df.MonthIndex <= 29)]

In [198]:
data.head(3)

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,...,promoratio,holidayratio,avgcustomers,avgsales,Sales2MonthsBack,Customers2MonthsBack,Sales3MonthsBack,Customers3MonthsBack,Sales4MonthsBack,Customers4MonthsBack
52,1,6,5592,580,1,0,0,0,2015,5,...,0.996361,2.289436,564.049936,4759.096031,4294.5,531.5,4467.5,527.0,4690.0,565.0
53,1,5,4656,537,1,0,0,0,2015,5,...,0.996361,2.289436,564.049936,4759.096031,4294.5,531.5,4467.5,527.0,4690.0,565.0
54,1,4,4111,474,1,0,0,0,2015,5,...,0.996361,2.289436,564.049936,4759.096031,4294.5,531.5,4467.5,527.0,4690.0,565.0


In [199]:
#Initialize h2o cluster and clear any previously saved info from the cluster
h2o.init(nthreads=-1,max_mem_size='6G')
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_192"; Java(TM) SE Runtime Environment (build 1.8.0_192-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.192-b12, mixed mode)
  Starting server from /Users/LiamRoberts/anaconda3/envs/DS1/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/sj/qvzvqx5n1qv59yg5v3pz2y400000gn/T/tmp5wep0tw6
  JVM stdout: /var/folders/sj/qvzvqx5n1qv59yg5v3pz2y400000gn/T/tmp5wep0tw6/h2o_LiamRoberts_started_from_python.out
  JVM stderr: /var/folders/sj/qvzvqx5n1qv59yg5v3pz2y400000gn/T/tmp5wep0tw6/h2o_LiamRoberts_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/Toronto
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.7
H2O cluster version age:,2 months and 10 days
H2O cluster name:,H2O_from_python_LiamRoberts_0afp26
H2O cluster total nodes:,1
H2O cluster free memory:,5.333 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [200]:
#Create h2o frame for training models
data = h2o.H2OFrame(python_obj=data)
valid = h2o.H2OFrame(python_obj=valid)

/Users/LiamRoberts/anaconda3/envs/DS1/lib/python3.6/site-packages/h2o/utils/shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [201]:
#Log transform sales data and assign X and y columns names
data['log_sales'] = data['Sales'].log()
valid['log_sales'] = valid['Sales'].log()

X_labels = [i for i in data.col_names if (i not in ['Sales','Customers','log_sales'])]
y_labels = 'log_sales'

In [172]:
#Create Model
model = H2ORandomForestEstimator(    
    ntrees=50,
    max_depth = 30,
    stopping_rounds = 4,
    stopping_tolerance = 1e-4)

#h2o supports .fit() to fit into the sklearn pipeline however recommends using .train() 
#and passing the full dataframe into the model
model.train(x=X_labels,y=y_labels,training_frame=data,validation_frame=valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [184]:
#Check Model Performance
display(model.score_history().tail(1))

,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
69,,2018-11-11 22:29:46,20 min 9.116 sec,89.0,0.103018,0.073134,0.010613,0.142242,0.110221,0.020233


In [ ]:
#This line will prevent the cluster from getting its memory too filled up if the notebook is run multiple times
h2o.cluster().shutdown()